In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import scipy.cluster.hierarchy as shc
import re

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


In [ ]:
try:
    from kneed import KneeLocator 
except:
    !pip install kneed
    from kneed import KneeLocator
try:
    from yellowbrick.cluster import KElbowVisualizer
except:
    !pip install -U yellowbrick
    from yellowbrick.cluster import KElbowVisualizer 

In [ ]:
try:
    import plotly.express as px
    import plotly.graph_objects as go
except:
    !pip install plotly
    import plotly.express as px
    import plotly.graph_objects as go

In [ ]:
data = pd.read_csv("/kaggle/input/bank-customer-segmentation/bank_transactions.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
def check(df):
    l=[]
    columns=df.columns
    for col in columns:
        dtypes=df[col].dtypes
        nunique=df[col].nunique()
        sum_null=df[col].isnull().sum()
        l.append([col,dtypes,nunique,sum_null])
    df_check=pd.DataFrame(l)
    df_check.columns=['column','dtypes','nunique','sum_null']
    return df_check
check(data)

In [ ]:
data.duplicated().sum()

In [ ]:
plt.style.use("fivethirtyeight")
chart=sb.countplot(x='CustGender',data=data,palette='rocket',order=data['CustGender'].value_counts().index)
plt.title("Frequency",fontsize='20',backgroundcolor='AliceBlue',color='magenta')

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(subset=['CustLocation','CustomerDOB','CustAccountBalance'],inplace=True)
data.shape

In [ ]:
data.fillna({
    "CustGender":"M"
}, inplace = True)

In [ ]:
data["CustomerDOB"]=pd.to_datetime(data["CustomerDOB"])
data["TransactionDate"]=pd.to_datetime(data["TransactionDate"])

data["CustomerAge"] = data["TransactionDate"].dt.year
-data["CustomerDOB"].dt.year

In [ ]:
data['TransactionDate1']=data['TransactionDate']
data['TransactionDate2']=data['TransactionDate']

In [ ]:
RMF_data = data.groupby("CustomerID").agg({
    "TransactionID" : "count",
    "CustGender" : "first",
    "CustLocation" : "first",
    "CustAccountBalance" : "mean",
    "TransactionAmount (INR)" : "mean",
    "CustomerAge" : "max",
    "TransactionDate2" : "max",
    "TransactionDate1" : "min",
    "TransactionDate" : "median",
})
RMF_data = RMF_data.reset_index()
RMF_data.head()

In [ ]:
RMF_data.drop(columns=['CustomerID'],inplace=True)

In [ ]:
RMF_data['Recency']=RMF_data['TransactionDate2']-RMF_data['TransactionDate1']

In [ ]:
RMF_data['Recency']=RMF_data['Recency'].astype(str)
RMF_data.head()

In [ ]:
RMF_data['Recency']=RMF_data['Recency'].apply(lambda x:re.search('\d+',x).group())
RMF_data['Recency']=RMF_data['Recency'].astype(int)

In [ ]:
def rep_0(i):
    if i==0:
        return 1
    else:
        return 1
RMF_data['Recency']=RMF_data['Recency'].apply(rep_0)

In [ ]:
RMF_data.drop(columns=['TransactionDate1',"TransactionDate2"],inplace=True)
RMF_data.head(3)

In [ ]:
RMF_data.rename(columns={'TransactionID':"Frequency"},inplace=True)

In [ ]:
lower_list=[]
upper_list=[]
num_list=[]
perc_list=[]
cols=['Frequency','CustAccountBalance','TransactionAmount (INR)','CustomerAge','Recency']
for i in cols:
    Q1 = RMF_data[i].quantile(0.25)
    Q3 = RMF_data[i].quantile(0.75)
    IQR = Q3 -Q1
    lower = Q1 -1.5 * IQR
    upper = Q3 +1.5 * IQR
    num=RMF_data[(RMF_data[i]<lower) | (RMF_data[i]>upper)].shape[0]
    perc = (num/RMF_data.shape[0])*100
    lower_list.append(lower)
    upper_list.append(upper)
    num_list.append(num)
    perc_list.append(round(perc,2))
dic={'lower' : lower_list, 'upper':upper_list, 'outliers' : num_list, 'Perc%' : perc_list}
outliers_df=pd.DataFrame(dic,index=['Frequency','CustAccountBalance','TransactionAmount (INR)','CustomerAge','Recency'])
outliers_df

EDA

In [ ]:
RMF_data.describe()

In [ ]:
RMF_data.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lr=LabelEncoder()
RMF_data['CustGender']=lr.fit_transform(RMF_data['CustGender'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
lr=LabelEncoder()
RMF_data['CustLocation']=lr.fit_transform(RMF_data['CustLocation'])

In [ ]:
plt.figure(figsize=(7,5))
correlation=RMF_data.corr()
sb.heatmap(correlation,vmin=None,vmax=0.8,cmap='rocket_r',annot=True,fmt='.1f',linecolor='white',cbar=True)

In [ ]:
RMF_data.reset_index(drop=True, inplace=True)
RMF_data.head()

In [ ]:
minmax = MinMaxScaler()
standard_scaler = StandardScaler()
RMF_scaled =RMF_data.copy()
RMF_scaled[["Frequency", "CustGender","Recency","CustLocation"]] = minmax.fit_transform(RMF_data[["Frequency", "CustGender","Recency","CustLocation"]])
RMF_scaled[["CustAccountBalance", "TransactionAmount (INR)","CustomerAge"]] = standard_scaler.fit_transform(RMF_data[["CustAccountBalance", "TransactionAmount (INR)","CustomerAge"]])

In [ ]:
RMF_scaled.head()

In [ ]:
RMF_scaled.info()

In [ ]:
RMF_scaled.drop(columns=['TransactionDate'],inplace=True)

In [ ]:
RMF_scaled.head()

In [ ]:
RMF_scaled.info()

In [ ]:
RMF_scaled=RMF_scaled.sample(n=7500,random_state=69).reset_index(drop=True)
RMF_scaled.head()

PCA

In [ ]:
trail_pca=PCA(n_components=4)
trail_pca.fit(RMF_scaled)
test=trail_pca.transform(RMF_scaled)
plt.plot(list(trail_pca.explained_variance_ratio_),'-o')
plt.title('Explained Variance ratio as function of pca component')
plt.ylabel('Explained variance ratio')
plt.xlabel('Components')
plt.show()

In [ ]:
trail_pca=PCA(n_components=7)
trail_pca.fit(RMF_scaled)
test=trail_pca.transform(RMF_scaled)
plt.plot(list(trail_pca.explained_variance_ratio_),'-o')
plt.title('Explained Variance ratio as function of pca component')
plt.ylabel('Explained variance ratio')
plt.xlabel('Components')
plt.show()

In [ ]:
trail_pca=PCA(n_components=3)
trail_pca.fit(RMF_scaled)
test=trail_pca.transform(RMF_scaled)
plt.plot(list(trail_pca.explained_variance_ratio_),'-o')
plt.title('Explained Variance ratio as function of pca component')
plt.ylabel('Explained variance ratio')
plt.xlabel('Components')
plt.show()

In [ ]:
RMF_pca=PCA(n_components=3)
reduced_features=RMF_pca.fit_transform(RMF_scaled)
x_principal=pd.DataFrame(reduced_features)
x_principal.columns=['p1','p2','p3']

In [ ]:
x_principal

KMeans

In [ ]:
# Elbow-curve
kmeans_set = {"init":"random", "max_iter":300, "random_state":42}
ssd = []
range_n_clusters = range(2,12)
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters,**kmeans_set)
    kmeans.fit(RMF_scaled)
    
    ssd.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters
ig, ax = plt.subplots(figsize=(10,5))
sb.lineplot(y=ssd, x=range_n_clusters, marker='o',ax=ax )
plt.title('The Elbow Method')
plt.xlabel('No of clusters "k"')
plt.ylabel('Inertia')
plt.show();

In [ ]:
WCSS =[]
silh_scores_kmeans =  []
db_scores_kmeans = []
for n_cluster in range(2,11):
    model_kmeans = KMeans(n_clusters = n_cluster)
    model_kmeans.fit(reduced_features)
    WCSS.append(model_kmeans.inertia_)
    silh_scores_kmeans.append(silhouette_score(reduced_features,model_kmeans.labels_))
    db_scores_kmeans.append(davies_bouldin_score(reduced_features,model_kmeans.labels_))

In [ ]:
silh_scores_kmeans

In [ ]:
db_scores_kmeans

In [ ]:
WCSS

In [ ]:
model_kmeans

In [ ]:
plt.style.use("fivethirtyeight")
k1=KneeLocator(range(2,11),WCSS, curve='convex', direction='decreasing')
plt.plot(range(2,11),WCSS, color='purple', marker='o',label='Elbow method')
plt.xticks(range(2,11))
plt.xlabel("Number of Clusters", labelpad=20)
plt.ylabel("Inertia",labelpad=20)
plt.title("Elbow Method")
plt.axvline(x=k1.elbow, color='black',label='axvline-fullheight',ls='--', linewidth=3)

In [ ]:
x_principal=x_principal.sample(n=1000,random_state=69).reset_index(drop=True)
x_principal.shape

In [ ]:
plt.title('Visualising the data')
Dendogram=shc.dendrogram((shc.linkage(x_principal,method='ward')))

In [ ]:
silh_scores_agglo = []
db_scores_agglo =[]
for n_cluster in range(2,11):
    model_agglo= AgglomerativeClustering(n_clusters=n_cluster)
    model_agglo.fit(x_principal)
    
    silh_scores_agglo.append(silhouette_score(x_principal, model_agglo.labels_))
    db_scores_agglo.append(davies_bouldin_score(x_principal, model_agglo.labels_))

In [ ]:
silh_scores_agglo

In [ ]:
db_scores_agglo

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(range(1,10),silh_scores_agglo, label="Silhouette Score")
plt.plot(range(1,10),db_scores_agglo, label="Davies Bouldin Score")
plt.xlabel("Number of Clusters")
plt.ylabel("Metrics")
plt.title("Metric Evaluvation for Agglomerative Clustering")
plt.legend()

In [ ]:
db = DBSCAN(eps=10,min_samples=1000).fit(x_principal)
core_samples_mask=np.zeros_like(db.labels_,dtype=bool)
core_samples_mask[db.core_sample_indices_]=True
labels=db.labels_

In [ ]:
x_principal['label']= labels
x_principal.head()

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D('p1','p2','p3', c='label', data=x_principal, cmap='viridis',linewidth=0.5)
ax.set_xlabel('p1')
ax.set_ylabel('p2')
ax.set_zlabel('p3')
plt.title("DBscan Clustering")
plt.legend()
plt.show()